# Summation of primes

[problem 10](https://projecteuler.net/problem=10)
> The sum of the primes below 10 is: 
> $$2 + 3 + 5 + 7 = 17$$
> Find the sum of all the primes below two million.

### [Sieve of Eratosthenes](http://en.wikipedia.org/wiki/Sieve_of_Eratosthenes)

Naive implementation that sieves until $\sqrt{n}$ is reached.  

This creates a set of all non prime numbers less than n that is used to check primality for numbers 2 to n.

In [1]:
def eratosthenes_set_update(n):
    """ Returns a list of primes < n """
    sieve = set()
    for i in range(2, int(n**0.5)+1):
        if i not in sieve:
            sieve.update(range(i*i, n, i))
    return [i for i in range(2, n) if i not in sieve]

for i in range(10):
    print (i, eratosthenes_set_update(i))
    
%timeit eratosthenes_set_update(10**7)
print(len(eratosthenes_set_update(10**7)))

0 []
1 []
2 []
3 [2]
4 [2, 3]
5 [2, 3]
6 [2, 3, 5]
7 [2, 3, 5]
8 [2, 3, 5, 7]
9 [2, 3, 5, 7]
1 loops, best of 3: 4.51 s per loop
664579


Only checking odd numbers increases performance significantly.  

This creates a set with all odd non prime numbers less than n. Iterative over odd numbers in the range and taking values that are not in the sieve gives us our list of primes.  

Note that 2 is added explicitly to the results.

In [2]:
def eratosthenes_set_update_odds(n):
    """ Returns a list of primes < n """
    sieve = set()
    for i in range(3, int(n**0.5)+1, 2):
        if i not in sieve:
            sieve.update(range(i*i, n, 2*i))
    return ([2] if n > 2 else []) + [i for i in range(3, n, 2) if i not in sieve]
    
for i in range(10):
    print (i, eratosthenes_set_update_odds(i))
    
%timeit eratosthenes_set_update_odds(10**7)
print(len(eratosthenes_set_update_odds(10**7)))

0 []
1 []
2 []
3 [2]
4 [2, 3]
5 [2, 3]
6 [2, 3, 5]
7 [2, 3, 5]
8 [2, 3, 5, 7]
9 [2, 3, 5, 7]
1 loops, best of 3: 2.31 s per loop
664579


Instead of updating a set, we can allocate a list to use as a sieve and update using slicing.

In [3]:
def eratosthenes_array_odds(n):
    """ Returns a list of primes < n """
    sieve = list(range(n))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[i]:
            sieve[i*i::i] = [0] * ((n-i*i-1)//i + 1)
    return ([2] if n > 2 else []) + [i for i in sieve[3::2] if i]

for i in range(10):
    print (i, eratosthenes_array_odds(i))
    
%timeit eratosthenes_array_odds(10**7)
print(len(eratosthenes_array_odds(10**7)))

0 []
1 []
2 []
3 [2]
4 [2, 3]
5 [2, 3]
6 [2, 3, 5]
7 [2, 3, 5]
8 [2, 3, 5, 7]
9 [2, 3, 5, 7]
1 loops, best of 3: 1.48 s per loop
664579


Another way to save memory is to use a list of booleans rather than the actual numbers.  

To get the list of primes we can iterate over the odds in range and check the index in the sieve.

In [4]:
def eratosthenes_bool_array_odds(n):
    """ Returns a list of primes < n """
    sieve = [True] * n
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[i]:
            sieve[i*i::i] = [0] * ((n-i*i-1)//i + 1)
    return ([2] if n > 2 else []) + [i for i in range(3, n, 2) if sieve[i]]

for i in range(10):
    print (i, eratosthenes_bool_array_odds(i))

%timeit eratosthenes_bool_array_odds(10**7)
print(len(eratosthenes_bool_array_odds(10**7)))

0 []
1 []
2 []
3 [2]
4 [2, 3]
5 [2, 3]
6 [2, 3, 5]
7 [2, 3, 5]
8 [2, 3, 5, 7]
9 [2, 3, 5, 7]
1 loops, best of 3: 871 ms per loop
664579


For even an even more efficient method, we can allocate an array for only the odd numbers.

In [5]:
def eratosthenes_bool_array_odd_half_sieve(n):
    """ Returns a list of primes < n """
    sieve = [True] * (n//2)
    end = len(sieve)
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[i//2]:
            start = i*i//2
            sieve[start::i] = [False] * ((end-start-1)//i + 1)
    return ([2] if n > 2 else []) + [2*i+1 for i in range(1, n//2) if sieve[i]]
    
for i in range(10):
    print (i, eratosthenes_bool_array_odd_half_sieve(i))
    
%timeit eratosthenes_bool_array_odd_half_sieve(10**7)
print(len(eratosthenes_bool_array_odd_half_sieve(10**7)))

0 []
1 []
2 []
3 [2]
4 [2, 3]
5 [2, 3]
6 [2, 3, 5]
7 [2, 3, 5]
8 [2, 3, 5, 7]
9 [2, 3, 5, 7]
1 loops, best of 3: 722 ms per loop
664579


### [Sieve of Sundaram](http://en.wikipedia.org/wiki/Sieve_of_Sundaram)

In [6]:
def sundaram(n):
    """ Returns a list of primes < n """
    from itertools import count
    half = n//2
    sieve = [True] * half
    for i in count(1):
        step  = 2*i + 1
        start = i * (step+1)
        if start >= half:
            break
        sieve[start::step] = [False] * ((half-start-1)//step + 1)
    return ([2] if n > 2 else []) + [2*k+1 for k in range(1, half) if sieve[k]]

for i in range(10):
    print(i, sundaram(i))

%timeit sundaram(10**7)
print(len(sundaram(10**7)))

0 []
1 []
2 []
3 [2]
4 [2, 3]
5 [2, 3]
6 [2, 3, 5]
7 [2, 3, 5]
8 [2, 3, 5, 7]
9 [2, 3, 5, 7]
1 loops, best of 3: 856 ms per loop
664579


## Solution

In [7]:
def euler10():
    return sum(eratosthenes_bool_array_odd_half_sieve(2*10**6))

%timeit euler10()
print(euler10())

10 loops, best of 3: 137 ms per loop
142913828922
